In [3]:
!pip install fastdtw numpy

    extract-msg (<=0.29.*)
                 ~~~~~~~^


In [1]:
import whisper
import librosa

path_audio = '../uploads/recording.wav'
model = whisper.load_model("base")  # Load the Whisper model
result = model.transcribe(path_audio, language='en', beam_size=5, best_of=3)  # Transcribe the audio

print(result)

/Users/ngocp/.pyenv/versions/3.10.3/lib/python3.10/site-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


{'text': ' Technically, it has a difficult transform how to communicate work and live, making our daily task more efficient when also very concerned about privacy and job replacement.', 'segments': [{'id': 0, 'seek': 0, 'start': 0.0, 'end': 8.0, 'text': ' Technically, it has a difficult transform how to communicate work and live, making our', 'tokens': [50364, 42494, 11, 309, 575, 257, 2252, 4088, 577, 281, 7890, 589, 293, 1621, 11, 1455, 527, 50764], 'temperature': 0.0, 'avg_logprob': -0.47807203020368305, 'compression_ratio': 1.34375, 'no_speech_prob': 0.05649280548095703}, {'id': 1, 'seek': 0, 'start': 8.0, 'end': 14.0, 'text': ' daily task more efficient when also very concerned about privacy and job replacement.', 'tokens': [50764, 5212, 5633, 544, 7148, 562, 611, 588, 5922, 466, 11427, 293, 1691, 14419, 13, 51064], 'temperature': 0.0, 'avg_logprob': -0.47807203020368305, 'compression_ratio': 1.34375, 'no_speech_prob': 0.05649280548095703}], 'language': 'en'}


1) no_speech_prob
- Logits: Raw scores from the model for different classes (e.g., speech, no speech).
- Softmax Function: Converts logits into probabilities.
- No Speech Probability: The probability for the "no speech" class after applying softmax.


$$
\text{no\_speech\_prob} = \sum_{j} e^{z_j} / e^{z_{\text{no\_speech}}}
$$


In [15]:
import nltk
nltk.download('cmudict')

[nltk_data] Downloading package cmudict to /Users/ngocp/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


True

In [32]:
from nltk.corpus import cmudict

# Load the CMU Pronouncing Dictionary
d = cmudict.dict()
print(f"Number of words in the CMU Pronouncing Dictionary: {len(d)}")
print(f"Pronunciation of 'hello': {d['hello']}")
# Get the pronunciation of a word
word = "hello"
# pronunciation = d[word.lower()][0]
# print(pronunciation)


Number of words in the CMU Pronouncing Dictionary: 123455
Pronunciation of 'hello': [['HH', 'AH0', 'L', 'OW1'], ['HH', 'EH0', 'L', 'OW1']]


In [58]:

def get_phonetic_transcription(word):
    """Returns the phonetic transcription of a word."""
    return d.get(word.lower())

def syllable_to_char_map(word, syllables):
    """Map each syllable to its corresponding characters in the word."""
    char_map = []
    start = 0
    for syllable in syllables:
        end = start + len(syllable)
        char_map.append(word[start:end])
        start = end
    return char_map

def highlight_syllables(expected_syllables, actual_syllables, word):
    """Highlight each syllable, red for wrong syllable, green for right syllable."""
    highlighted = []
    char_map = syllable_to_char_map(word, actual_syllables)
    for exp_syl, act_syl, chars in zip(expected_syllables, actual_syllables, char_map):
        print(f"highlight_syllables: {exp_syl} -> {act_syl} -> {chars}")
        if exp_syl == act_syl:
            highlighted.append(f'<span style="color: green;">{chars}</span>')
        else:
            highlighted.append(f'<span style="color: red;">{chars}</span>')
    return highlighted

def compare_phonetic_transcriptions(expected, actual):
    """Compare the phonetic transcriptions of the expected and actual words."""
    feedback = []
    for exp_word, act_word in zip(expected.split(), actual.split()):
        exp_phonetic = get_phonetic_transcription(exp_word)
        act_phonetic = get_phonetic_transcription(act_word)
        print(f"Expected: {exp_word} -> {exp_phonetic} -> {act_word}")
        if exp_phonetic and act_phonetic:
            exp_syllables = [syllable for phoneme in exp_phonetic for syllable in phoneme]
            act_syllables = [syllable for phoneme in act_phonetic for syllable in phoneme]
            highlighted_syllables = highlight_syllables(exp_syllables, act_syllables, act_word)
            feedback.append({'word': act_word, 'highlighted_syllables': ''.join(highlighted_syllables)})
        else:
            feedback.append({'word': act_word, 'highlighted_syllables': f'<span style="color: red;">{act_word}</span>'})
    
    return feedback

In [59]:
user_text = "We are to establish an environment, just support sustainable practices."
reference_text = "We ought to establish an environment that supports sustainable practices."

feedback = compare_phonetic_transcriptions(user_text, reference_text)

for item in feedback:
    print(item)

Expected: We -> [['W', 'IY1']] -> We
highlight_syllables: W -> W -> W
highlight_syllables: IY1 -> IY1 -> e
Expected: are -> [['AA1', 'R'], ['ER0']] -> ought
highlight_syllables: AA1 -> AO1 -> oug
highlight_syllables: R -> T -> h
Expected: to -> [['T', 'UW1'], ['T', 'IH0'], ['T', 'AH0']] -> to
highlight_syllables: T -> T -> t
highlight_syllables: UW1 -> UW1 -> o
highlight_syllables: T -> T -> 
highlight_syllables: IH0 -> IH0 -> 
highlight_syllables: T -> T -> 
highlight_syllables: AH0 -> AH0 -> 
Expected: establish -> [['IH0', 'S', 'T', 'AE1', 'B', 'L', 'IH0', 'SH'], ['IY0', 'S', 'T', 'AE1', 'B', 'L', 'IH0', 'SH']] -> establish
highlight_syllables: IH0 -> IH0 -> est
highlight_syllables: S -> S -> a
highlight_syllables: T -> T -> b
highlight_syllables: AE1 -> AE1 -> lis
highlight_syllables: B -> B -> h
highlight_syllables: L -> L -> 
highlight_syllables: IH0 -> IH0 -> 
highlight_syllables: SH -> SH -> 
highlight_syllables: IY0 -> IY0 -> 
highlight_syllables: S -> S -> 
highlight_syllable

In [17]:
def syllable_count(word):
    """Returns the number of syllables in a word."""
    if word.lower() in d:
        return [len(list(y for y in x if y[-1].isdigit())) for x in d[word.lower()]][0]
    else:
        return len([char for char in word if char in 'aeiouy'])

In [ ]:
import subprocess

def get_ipa_transcription(text):
    result = subprocess.run(
        ['espeak-ng', '-x', text, '--ipa'],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )
    return result.stdout.strip()


In [ ]:
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

def compare_phonetic_sequences(seq1, seq2):
    # Convert IPA sequences to numerical representation if needed
    # For simplicity, this example uses string comparison
    distance, path = fastdtw(seq1, seq2, dist=euclidean)
    return distance, path

In [ ]:
def highlight_discrepancies(user_text, user_phonetic, expected_phonetic):
    discrepancies = []
    for i, (user, expected) in enumerate(zip(user_phonetic, expected_phonetic)):
        if user != expected:
            discrepancies.append((i, user, expected))
    return discrepancies

# Example usage
user_text = "Hello, how are you?"
expected_text = "Hello, how are you?"

path_user_voice = "../uploads/recording.wav"
user_transcription = transcribe_audio(path_user_voice)
print('user_transcription:', user_transcription)

expected_phonetic = get_ipa_transcription(expected_text)
user_phonetic = get_ipa_transcription(user_transcription)

print("Expected phonetic transcription:", expected_phonetic, user_phonetic)
distance, path = compare_phonetic_sequences(user_phonetic.split(), expected_phonetic.split())
discrepancies = highlight_discrepancies(user_transcription, user_phonetic.split(), expected_phonetic.split())

print("Discrepancies:", discrepancies)


/Users/ngocp/.pyenv/versions/3.10.3/lib/python3.10/site-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


user_transcription:  secret is simple sentence.
Expected phonetic transcription: həlˈəʊ
hˈaʊ ɑː juː sˈiːkɹɪt ɪz sˈɪmpəl sˈɛntəns


ValueError: could not convert string to float: 'sˈiːkɹɪt'